# Maximum Likelihood Estimation

We will apply the maximum likelihood estimation to try to predict the angles theta and phi from the bloch sphere.

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
# ============================================================
# 1. Config
# ============================================================
CSV_PATH = "../data/qst_regression_dataset.csv"
N_SHOTS = 200  # per basis X, Y, Z

In [ ]:
# ============================================================
# 2. Helpers: Bloch vector and conversions
# ============================================================
def bloch_from_angles(theta, phi):
    """
    Given (theta, phi), return Bloch vector (nx, ny, nz).
    """
    nx = np.sin(theta) * np.cos(phi)
    ny = np.sin(theta) * np.sin(phi)
    nz = np.cos(theta)
    return nx, ny, nz

def angles_from_bloch(nx, ny, nz):
    """
    Given Bloch vector (nx, ny, nz) on the sphere, return (theta, phi).
    """
    # First we normalize (just in case)
    norm = np.sqrt(nx ** 2 + ny ** 2 + nz ** 2)
    nx /= norm
    ny /= norm
    nz /= norm
    
    # Calculate theta and phi
    theta = np.arccos(nz)
    phi = np.arctan(ny/nx)
    return theta, phi
    

In [4]:
df = pd.read_csv("../data/qst_regression_dataset.csv")

df.head()

,X_mean,Y_mean,Z_mean,theta_ideal,phi_ideal,cos_phi_ideal,sin_phi_ideal,X_ideal,Y_ideal,Z_ideal
0,0.93,-0.35,-0.30,1.824427,5.973514,0.952434,-0.304745,0.921963,-0.294996,-0.250920
1,0.72,0.50,-0.65,2.273101,0.557334,0.848668,0.528926,0.647835,0.403758,-0.645979
2,0.14,-1.00,-0.01,1.593512,4.840613,0.127873,-0.991791,0.127840,-0.991535,-0.022714
3,0.34,0.87,0.41,1.242172,1.163618,0.396020,0.918242,0.374828,0.869104,0.322741
4,-0.22,0.12,0.99,0.219199,2.852576,-0.958525,0.285009,-0.208429,0.061975,0.976072


In [ ]:
# calculer les probas de 1 en X, Y, Z
def likelihood_calculator(proba_1, number, shots):
    return proba_1 ** number * (1 - proba_1) ** (shots - number)

# Calcul des probabilités à partir des résultats des moyennes
df["probaX+"] = (1 + df["X_mean"]) / 2.0
df["probaY+"] = (1 + df["Y_mean"]) / 2.0
df["probaZ+"] = (1 + df["Z_mean"]) / 2.0

df["numberX"] = df["probaX+"] * N_SHOTS
df["numberY"] = df["probaY+"] * N_SHOTS
df["numberZ"] = df["probaZ+"] * N_SHOTS

df["likelihood"] = (
    likelihood_calculator(df["probaX+"], df["numberX"], N_SHOTS) * 
    likelihood_calculator(df["probaY+"], df["numberY"], N_SHOTS) * 
    likelihood_calculator(df["probaZ+"], df["numberZ"], N_SHOTS)
)

initial_theta_estimation, initial_phi_estimation = angles_from_bloch(X_mean, Y_mean, Z_mean)

df.head()

,X_mean,Y_mean,Z_mean,theta_ideal,phi_ideal,cos_phi_ideal,sin_phi_ideal,X_ideal,Y_ideal,Z_ideal,probaX+,probaY+,probaZ+,numberX,numberY,numberZ,likelihood
0,0.93,-0.35,-0.30,1.824427,5.973514,0.952434,-0.304745,0.921963,-0.294996,-0.250920,0.965,0.325,0.350,193.0,65.0,70.0,6.518911e-125
1,0.72,0.50,-0.65,2.273101,0.557334,0.848668,0.528926,0.647835,0.403758,-0.645979,0.860,0.750,0.175,172.0,150.0,35.0,5.043613e-125
2,0.14,-1.00,-0.01,1.593512,4.840613,0.127873,-0.991791,0.127840,-0.991535,-0.022714,0.570,0.000,0.495,114.0,0.0,99.0,2.794889e-120
3,0.34,0.87,0.41,1.242172,1.163618,0.396020,0.918242,0.374828,0.869104,0.322741,0.670,0.935,0.705,134.0,187.0,141.0,2.186517e-129
4,-0.22,0.12,0.99,0.219199,2.852576,-0.958525,0.285009,-0.208429,0.061975,0.976072,0.390,0.560,0.995,78.0,112.0,199.0,3.980493e-121
